In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import random

In [2]:
structures = pd.read_csv("structures.csv")

In [3]:
structures.shape

(2358657, 6)

In [4]:
test = pd.read_csv("test.csv")

In [5]:
test.shape

(2505542, 5)

In [6]:
inter_test = test.merge(structures, how='inner', left_on=['molecule_name','atom_index_0'], 
                right_on=['molecule_name','atom_index'])
loaded_test = inter_test.merge(structures, how='inner', left_on=['molecule_name','atom_index_1'], 
                right_on=['molecule_name','atom_index'])
loaded_test_dropped_cols = loaded_test.drop(['atom_index_x','atom_index_y'], axis=1)

In [8]:
type_field = loaded_test_dropped_cols["type"]

In [9]:
lb = LabelBinarizer()

In [10]:
type_field_bin = lb.fit_transform(type_field)

In [12]:
coordinates_df = loaded_test_dropped_cols[["x_x", "y_x", "z_x", "x_y", "y_y", "z_y"]]

In [13]:
coordinates_array = coordinates_df.as_matrix()

/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [14]:
final_array_t = np.concatenate((type_field_bin, coordinates_array), axis = 1)

In [15]:
molecule_id = np.zeros(len(final_array_t))
prev_molecule = "garbage"
i = 0
j = 0
for each_row in loaded_test_dropped_cols.iterrows():
    new_molecule = each_row[1][1]
    if prev_molecule != new_molecule:
        i = i + 1
    molecule_id[j] = i
    prev_molecule = new_molecule
    j = j + 1

In [16]:
molecule_id_T = molecule_id.reshape((len(molecule_id),1))

In [17]:
final_array_t = np.concatenate((molecule_id_T, final_array_t), axis = 1)

In [18]:
final_array_t.shape

(2505542, 15)

In [19]:
final_array_t[-1][0]

45772.0

In [20]:
def convert_to_superarray(final_array_2):
    superarray = np.zeros((45772, 135, 15, 1))
    begin = 0
    end = 0
    mole = 0
    while ((mole <= 45770) and (begin <= 2505541) and (end <= 2505541)):
        while final_array_2[end][0] <= final_array_2[begin][0]:
            end = end + 1
        length = end - begin
        X_batch_temp_t = final_array_2[begin:end, :].copy()
        up = (135 - length) // 2
        if (135 - length) % 2 == 1:
            down = ((135 - length) // 2) + 1
        else:
            down = ((135 - length) // 2)
        X_batch_temp_2 = np.pad(X_batch_temp_t, ((up, down), (0, 0)), 'constant')
        X_batch_temp_3 = X_batch_temp_2.reshape((135, 15, 1))
        superarray[mole] = X_batch_temp_3
        mole = mole + 1
        begin = end
    return superarray

In [21]:
superarray = convert_to_superarray(final_array_t)

In [22]:
superarray.shape

(45772, 135, 15, 1)

In [28]:
X_test = superarray[:, :, 1:15, :].copy()

In [29]:
X_test.shape

(45772, 135, 14, 1)

In [37]:
X = tf.placeholder(tf.float32, shape=[None, 135, 14, 1])
y = tf.placeholder(tf.float32, shape=[None, 135])
training_flag = 1
batch_size = tf.placeholder(tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size).repeat()
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

In [38]:
with tf.name_scope("cnn"):
    X_normalized = tf.layers.batch_normalization(features, training = training_flag)
    conv1 = tf.layers.conv2d(X_normalized, 3, [76, 9])
    conv1_normalized = tf.layers.batch_normalization(conv1, training = training_flag)
    conv1_out = tf.nn.relu(conv1_normalized)
    conv2 = tf.layers.conv2d(conv1_out, 6, [36, 3])
    conv2_normalized = tf.layers.batch_normalization(conv2, training = training_flag)
    conv2_out = tf.nn.relu(conv2_normalized)
    conv3 = tf.layers.conv2d(conv2_out, 10, [16, 2])
    conv3_normalized = tf.layers.batch_normalization(conv3, training = training_flag)
    conv3_out = tf.nn.relu(conv3_normalized)
    flat = tf.layers.flatten(conv3)
    output = tf.layers.dense(flat, 135)

In [39]:
with tf.name_scope("loss"):
    l2_loss = tf.losses.mean_squared_error(labels, output)

Instructions for updating:
Use tf.cast instead.


In [40]:
learning_rate = 0.01
with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer().minimize(l2_loss)
    #training_op = optimizer.minimize(l2_loss)

In [41]:
with tf.name_scope("estimated_MAE"):
    MAE_hat = tf.math.reduce_mean(tf.metrics.mean_absolute_error(labels, output))
    log_MAE_hat = tf.math.log(MAE_hat)

In [42]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [43]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # Restore variables from disk.
    saver.restore(sess, "./my_model_final2.ckpt")
    print("Model restored.")
    # Check the values of the variables
    sess.run(iter.initializer, feed_dict={ X: X_test, y: np.zeros((45772, 135)), batch_size: X_test.shape[0]})
    out = sess.run(output)
    #print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))

INFO:tensorflow:Restoring parameters from ./my_model_final2.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key batch_normalization_10/beta not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-42-0941fea4cac3>:2) ]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_insta